In [12]:
import numpy as np
import glob
import pandas as pd
import os
import dill


%matplotlib osx
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from import_raw import get_raw_data
from fill_and_shape import reshape_and_interp
from align_coords import align
from Piecewise_Smoothing import return_smoothed
from kinematics_fn import kins_fn


# Section 1: Importing new data

In [13]:
cwd = os.getcwd()
path = cwd + '/MOCAP files/*'
Files = glob.glob(path)
Files.sort()

## STEP 1: import CSVs and associated metadata
### Also add in new key to hold gap size data

In [14]:
all_imported = []
for each in Files:
    raw_dict = get_raw_data(each)
    all_imported.append(raw_dict)

/Users/grahmich/Desktop/Code/import_raw.py:60: RuntimeWarning: Mean of empty slice
  target = np.nanmean(branch[:,1,:],0)
/Users/grahmich/Desktop/Code/import_raw.py:59: RuntimeWarning: Mean of empty slice
  origin = np.nanmean(branch[:,0,:],0)
/Users/grahmich/Desktop/Code/import_raw.py:60: RuntimeWarning: Mean of empty slice
  target = np.nanmean(branch[:,1,:],0)
/Users/grahmich/Desktop/Code/import_raw.py:59: RuntimeWarning: Mean of empty slice
  origin = np.nanmean(branch[:,0,:],0)
/Users/grahmich/Desktop/Code/import_raw.py:59: RuntimeWarning: Mean of empty slice
  origin = np.nanmean(branch[:,0,:],0)
/Users/grahmich/Desktop/Code/import_raw.py:60: RuntimeWarning: Mean of empty slice
  target = np.nanmean(branch[:,1,:],0)
/Users/grahmich/Desktop/Code/import_raw.py:60: RuntimeWarning: Mean of empty slice
  target = np.nanmean(branch[:,1,:],0)
/Users/grahmich/Desktop/Code/import_raw.py:60: RuntimeWarning: Mean of empty slice
  target = np.nanmean(branch[:,1,:],0)
/Users/grahmich/Desktop/

In [15]:
#use motion capture locations of target and origin ends to calculate gap size

for i in np.arange(len(all_imported)):
    
    te = all_imported[i]['te']/1000.0 #positions recorded in mm, change to m
    oe = all_imported[i]['oe']/1000.0 
    gap = np.linalg.norm(te-oe)  
    
    svl = all_imported[i]['svl']/100.0 #svl is recorded in cm, change to m 
    gap_size = gap/svl*100 #svl
    
    all_imported[i]['gs_m'] = gap
    all_imported[i]['gs_%'] = gap_size

In [16]:
#manually enter gap sizes for trials where a branch marker did not record correctly.
trial_ns = [all_imported[i]['tn'] for i in np.arange(len(all_imported))]
tn2fx = [57,85,93,94,191,192,193,194] #relevant trial numbers 
edits = [trial_ns.index(each) for each in tn2fx] #index in the lists of those trials
sizes = [0.310,0.608,0.667,0.710,0.490,0.530,0.530,0.530] #corresponding gap sizes for those trials.

strng = "Absolute gap size entered manually based on lab notebook notes"

for i in np.arange(len(edits)):
    n = edits[i]
    all_imported[n]['Notes'] = strng
    all_imported[n]['gs_m'] = sizes[i]
    svl = all_imported[n]['svl']/100.0
    all_imported[n]['gs_%'] = sizes[i]/svl*100

## STEP 2: reshape and adjust for missing markers.


In [22]:
shaped_snakes = []
dropped_marks = []
new_mark_spaces = []
frs = []

files = list(range(0,len(all_imported)))

for s in files: 
    dictionary = all_imported[s]
    snake = dictionary['raw']
    mark_s = dictionary['ms']
    frate = dictionary['fr']
    n = 0.2    
    
    sn, drops, new_marks = reshape_and_interp(snake,mark_s,n,frate)
    
    shaped_snakes.append(sn) #the position data reshaped and interpolated
    dropped_marks.append(drops) #saving positions of dropped markers; useful for diagnostics but not otherwise used. 
    new_mark_spaces.append(new_marks) #the new list of marker spacings given any dropped markers.
    frs.append(frate) #list of frame rates. 
    
    #saving updated metadata
    all_imported[s]['cms'] = new_marks #save new marker spacings in the trial metadata dictionary.

## STEP 3: rotate coordinate system to align with vector pointing from origin to target.

In [30]:
aligned_snake = []
angles = []

for i in np.arange(len(all_imported)):
    #print(all_imported[i]['tn'])
    sn = shaped_snakes[i]
    dictionary = all_imported[i]
    origin_end = dictionary['oe']
    target_end = dictionary['te']
    
    #align and make origin_end origin of coordinate system
    (snake,theta,te) = align(sn, origin_end, target_end)
    target_end = (target_end-origin_end)
    
    aligned_snake.append(snake)
    angles.append(theta) #keeping track of how much the calibrated coordinate system deviated from the desired coordinate system. 
    
    #update metadata
    all_imported[i]['nte'] = target_end #new target end with origin end as coordinate system origin.
    
    

no target marker
no branch markers
no origin marker
no origin marker
no target marker
no target marker
no target marker
no target marker


In [8]:
#Check: graph of all the data (unsmoothed, but small missing spots filled in)
col = 7
row = 26

fig, ax = plt.subplots(col,row)

gs1 = gridspec.GridSpec(col, row)
gs1.update(wspace=0.025, hspace=0.05) # set the spacing between axes. 

for i in np.arange(col):
    for t in np.arange(row):
        n = i+col*t
        #print(n)
        ax[i,t].set_title(str(all_imported[n]['tn']),size=5)
        ax[i,t].plot(aligned_snake[n][:,:,0])
        ax[i,t].axis('off')
fig.suptitle('all data (n=182), unsmoothed')

Text(0.5, 0.98, 'all data (n=182), unsmoothed')

## STEP 4: smooth the data and create kinematics files

In [9]:
p = 10000.0

sm_pos = []
sm_vel = []
sm_acc = []
d_times = []

for sn in np.arange(len(aligned_snake)):
    print(sn) #progress tracker. This loop takes a long time.
    snake = aligned_snake[sn]
    mark_s = new_mark_spaces[sn]
    dms = dropped_marks[sn]
    frate = frs[sn]
    
    new_snake = np.zeros_like(snake)
    new_vs = np.zeros_like(snake)
    new_as = np.zeros_like(snake)
    new_ts = np.zeros_like(snake)

    for m in np.arange(len(mark_s)): #for each marker
        for dim in np.arange(3): #for each dimension
            test = snake[:,m,dim]
            conts, smoothie = return_smoothed(test,p)
            smooth_kins = kins_fn(conts, smoothie, frate)
            times = np.arange(len(snake[:,0,0]))/frate #make a list of all the time points
            
            #The kins function only returns a row for the defined sections of data. In order to preserve the gaps,
            #must go through and match the data in the kins array to the corresponding row in the complete 
            #matrix, that has all the timepoints (including remaining gaps) and fill in corresponding data.

            for ts in np.arange(len(smooth_kins[:,0])):
                ind = np.where(times == smooth_kins[ts,0])[0][0]
                new_snake[ind,m,dim] = smooth_kins[ts,1]
                new_vs[ind,m,dim] = smooth_kins[ts,2]
                new_as[ind,m,dim] = smooth_kins[ts,3]
                new_ts[ind,m,dim] = smooth_kins[ts,0]
                
    new_snake[new_snake==0] = np.nan
    new_vs[new_vs==0] = np.nan
    new_as[new_as==0] = np.nan
    new_ts[new_ts==0] = np.nan
    
    sm_pos.append(new_snake)
    sm_vel.append(new_vs)
    sm_acc.append(new_as)
    d_times.append(new_ts)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


In [11]:
#Final review: graph of all the smoothed position data.
col = 7
row = 26

fig, ax = plt.subplots(col,row)

gs1 = gridspec.GridSpec(col, row)
gs1.update(wspace=0.025, hspace=0.05) # set the spacing between axes. 

for i in np.arange(col):
    for t in np.arange(row):
        n = t*col+i
#         print(n)
        ax[i,t].set_title(str(all_imported[n]['tn']),size=5)
        ax[i,t].plot(sm_pos[n][:,:,0])
        ax[i,t].axis('off')
fig.suptitle('all data (n=182), smoothed')

Text(0.5, 0.98, 'all data (n=182), smoothed')

In [11]:
#save the processed position/velocity/acceleration/time data, and the trial metadata.

with open('smooth_pos_v8.pkl', 'wb') as var:
    dill.dump(sm_pos,var)
    
with open('smooth_vel_v8.pkl', 'wb') as var:
    dill.dump(sm_vel, var)

with open('smooth_acc_v8.pkl', 'wb') as var:
    dill.dump(sm_acc, var)
    
with open('smooth_times_v8.pkl', 'wb') as var:
    dill.dump(d_times, var)
    
with open('raw_dictionaries_v9.pkl', 'wb') as var:
    dill.dump(all_imported, var)
